In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from tensorflow.keras import callbacks
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten,
    Dense, Dropout, BatchNormalization, Activation, Input
)
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom


In [ ]:
#using generator function of keras
directory_train = "/Users/rishikumar/ML_Projects/Emotion_Detector/train"

train_ds = keras.utils.image_dataset_from_directory(
    directory = directory_train,
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="grayscale",
    batch_size=32,
    image_size=(48, 48),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format=None,
    format="tf",
    verbose=True,
)

In [ ]:
directory_test = "/Users/rishikumar/ML_Projects/Emotion_Detector/test"

test_ds = keras.utils.image_dataset_from_directory(
    directory = directory_test,
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="grayscale",
    batch_size=32,
    image_size=(48, 48),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format=None,
    format="tf",
    verbose=True,
)

In [ ]:
#Normalising the inputs
def normalize(image, label):
    image = image / 255.0
    return image, label
train_ds = train_ds.map(normalize)
test_ds = test_ds.map(normalize)

In [ ]:
#Creating CNN Model
model = Sequential()

# Block 1
model.add(Conv2D(32, (3,3), activation='relu', padding='same',input_shape=(48,48,1)))
model.add(MaxPooling2D((2,2)))

# Block 2
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D((2,2)))

# Block 3
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D((2,2)))

# Classifier
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'adam' , loss = 'sparse_categorical_crossentropy' , metrics = ['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

def get_early_stopping():
    return EarlyStopping(
        monitor='val_loss',  
        patience=3,              # wait 3 epochs
        restore_best_weights=True
    )


In [ ]:
early_stop = get_early_stopping()

history = model.fit(
    train_ds,
    epochs=30,            
    validation_data=test_ds,
    callbacks=[early_stop]
)

In [ ]:
data_augmentation = Sequential([
    RandomFlip("horizontal"),
    RandomRotation(0.05),
    RandomZoom(0.05),
])

In [ ]:
#Creating Modified CNN Model
model = Sequential()

model.add(Input(shape=(48,48,1)))  
model.add(data_augmentation)

model.add(Conv2D(32, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(128, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(7, activation='softmax'))


In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.summary()

In [ ]:
model.fit(
    train_ds,
    epochs=30,            
    validation_data=test_ds,
    callbacks=[early_stop]
)

In [ ]:
#Lets test this on a random internet photo
import requests
from PIL import Image
from io import BytesIO

url = "https://images.pexels.com/photos/220453/pexels-photo-220453.jpeg"

response = requests.get(url)
img = Image.open(BytesIO(response.content)).convert("L")  # grayscale

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(img, cmap='gray')
plt.axis('off')
plt.title("Input Image")
plt.show()

In [ ]:
import numpy as np

img = img.resize((48, 48))
img_array = np.array(img) / 255.0      # normalize
img_array = np.expand_dims(img_array, axis=-1)  # (48,48,1)
img_array = np.expand_dims(img_array, axis=0)   # (1,48,48,1)

In [ ]:
class_names = [
    'Angry',
    'Disgust',
    'Fear',
    'Happy',
    'Neutral',
    'Sad',
    'Surprise'
]

In [ ]:
pred = model.predict(img_array)
predicted_class = class_names[np.argmax(pred)]

print("Predicted Emotion:", predicted_class)